# Diplodatos Kaggle Competition

We present this peace of code to create the baseline for the competition, and as an example of how to deal with these kind of problems. The main goals are that you:

1. Learn
1. Try different models and see which one fits the best the given data
1. Get a higher score than the given one in the current baseline example
1. Try to get the highest score in the class :)

In [1]:
# Import the required packages
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Read the *original* dataset...

In [2]:
def transform_data(train_data_fname, test_data_fname):
    # Import the required packages
    import os
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns

    
    df_train = pd.read_csv(train_data_fname)
    df_train['is_train_set'] = 1
    df_test = pd.read_csv(test_data_fname)
    df_test['is_train_set'] = 0


    # we  get the TripType for the train set. To do that, we group by VisitNumber and
    # then we get the max (or min or avg)
    y = df_train.groupby(["VisitNumber", "Weekday"], as_index=False).max().TripType


    # we remove the TripType now, and concat training and testing data
    # the concat is done so that we have the same columns for both datasets
    # after one-hot encoding
    df_train = df_train.drop("TripType", axis=1)
    df = pd.concat([df_train, df_test])

    # the next three operations are the ones we have just presented in the previous lines
    #Here we wanted to add these two collumns for a better accuracy of the model. but we cannot resolve how to fit with the memory.

    # one-hot encoding
    df = pd.get_dummies(df, columns=["DepartmentDescription"], dummy_na=True)
    df = pd.get_dummies(df, columns=["FinelineNumber"], dummy_na=True)
    #df = pd.get_dummies(df, columns=["Upc"], dummy_na=True)

    # drop the columns we won't use (it may be good to use them somehow)
    df = df.drop(["Upc", "FinelineNumber"], axis=1)

    df.loc[df.ScanCount < 0, 'Return'] = 1
    df.loc[df.ScanCount > 0, 'Return'] = 0

    
    # now we add the groupby values
    df = df.groupby(["VisitNumber", "Weekday"], as_index=False).sum()
    
    # finally, we do one-hot encoding for the Weekday
    df = pd.get_dummies(df, columns=["Weekday"], dummy_na=True)
    
    #df_train = df_train.join(dummy_data)
    #df_test = df_test.join(dummy_data)
    
    # get train and test back
    df_train = df[df.is_train_set != 0] 
    df_test = df[df.is_train_set == 0]
    
    X = df_train.drop(["is_train_set"], axis=1)
    yy = None
    XX = df_test.drop(["is_train_set"], axis=1)

    return X, y, XX, yy

Load the data...

In [3]:
X, y, XX, yy = transform_data("~/diplo/clases/AprendizajeSupervisado/practico/data/train.csv", "~/diplo/clases/AprendizajeSupervisado/practico/data/test.csv")

NameError: name &#39;transform_data2&#39; is not defined

Create the model and evaluate it

In [23]:
# split training dataset into train and "validation" 
# (we won't be using validation set in this example, because of the cross-validation;
# but it could be useful for you depending on your approach)
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

In [24]:

#dtrain = xgb.DMatrix(np.asarray(X_train), label = np.asarray(y_train))
#dtest = xgb.DMatrix(np.asarray(X_valid), label = np.asarray(y_valid))



In [25]:
# fit model
model = XGBClassifier()
model.fit(X_train, y_train)

KeyboardInterrupt: 

In [11]:
y_pred = model.predict(X_valid)
predictions = [round(value) for value in y_pred]

In [13]:
accuracy = accuracy_score(y_valid, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 71.24%


In [16]:
yy = model.predict(XX)

In [17]:
len(yy),len(predictions)

(28645, 20109)

**And finally**, we predict the unknown label for the testing set

In [19]:
submission = pd.DataFrame(list(zip(XX.VisitNumber, yy)), columns=["VisitNumber", "TripType"])

In [20]:
submission.to_csv("../data/submission.csv", header=True, index=False)